In [1]:
# ===============================
# NHIỆM VỤ 1: ĐỌC & LÀM SẠCH DỮ LIỆU
# ===============================

import pandas as pd
import numpy as np
import os

# -------------------------------
# 1. ĐỌC DỮ LIỆU THÔ
# notebook nằm trong thư mục task03_groupby
# -------------------------------
patient = pd.read_csv("../data_raw/patient_info.csv", encoding="utf-8")
reaction = pd.read_csv("../data_raw/reaction_log.csv", encoding="utf-8")
vaccination = pd.read_csv("../data_raw/vaccination_record.csv", encoding="utf-8")

# -------------------------------
# 2. KHẢO SÁT CẤU TRÚC DỮ LIỆU
# -------------------------------
for name, df in {
    "PATIENT": patient,
    "REACTION": reaction,
    "VACCINATION": vaccination
}.items():
    print(f"\n{name}")
    print("Số dòng, số cột:", df.shape)
    print("Kiểu dữ liệu:")
    print(df.dtypes)

# -------------------------------
# 3. CHUẨN HOÁ DỮ LIỆU PHÂN LOẠI
# -------------------------------

# gender → M / F
patient["gender"] = (
    patient["gender"]
    .astype(str)
    .str.strip()
    .str.upper()
    .replace({"MALE": "M", "FEMALE": "F"})
)

# age_group
patient["age_group"] = (
    patient["age_group"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# vaccine_type
vaccination["vaccine_type"] = (
    vaccination["vaccine_type"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# -------------------------------
# 4. CHUẨN HOÁ SỐ & NGÀY THÁNG
# -------------------------------

# dose_number → số nguyên
vaccination["dose_number"] = (
    pd.to_numeric(vaccination["dose_number"], errors="coerce")
    .astype("Int64")
)

# ngày tiêm
vaccination["vaccination_date"] = pd.to_datetime(
    vaccination["vaccination_date"], errors="coerce"
)

# ngày báo cáo phản ứng
reaction["report_date"] = pd.to_datetime(
    reaction["report_date"], errors="coerce"
)

# -------------------------------
# 5. XỬ LÝ DỮ LIỆU THIẾU
# -------------------------------
patient = patient.dropna(subset=["patient_id"])

vaccination = vaccination.dropna(
    subset=["patient_id", "vaccination_date"]
)

reaction = reaction.dropna(
    subset=["patient_id", "report_date"]
)

# -------------------------------
# 6. KIỂM TRA LOGIC
# -------------------------------

# ngày báo cáo không được trước ngày tiêm
reaction = reaction.merge(
    vaccination[["patient_id", "vaccination_date"]],
    on="patient_id",
    how="left"
)

reaction = reaction[
    reaction["report_date"] >= reaction["vaccination_date"]
]

reaction = reaction.drop(columns=["vaccination_date"])

# liều tiêm trùng
vaccination = vaccination.drop_duplicates(
    subset=["patient_id", "dose_number"]
)

# -------------------------------
# 7. ĐỒNG BỘ patient_id
# -------------------------------
common_ids = (
    set(patient["patient_id"])
    & set(vaccination["patient_id"])
    & set(reaction["patient_id"])
)

patient = patient[patient["patient_id"].isin(common_ids)]
vaccination = vaccination[vaccination["patient_id"].isin(common_ids)]
reaction = reaction[reaction["patient_id"].isin(common_ids)]

# -------------------------------
# 8. XUẤT FILE DỮ LIỆU SẠCH
# -------------------------------
os.makedirs("data_clean", exist_ok=True)

patient.to_csv("data_clean/patient_clean.csv", index=False)
vaccination.to_csv("data_clean/vaccination_clean.csv", index=False)
reaction.to_csv("data_clean/reaction_clean.csv", index=False)

print("\n✅ HOÀN THÀNH NHIỆM VỤ 1")
print("📁 Các file đã được lưu trong thư mục data_clean/")



PATIENT
Số dòng, số cột: (40, 3)
Kiểu dữ liệu:
patient_id    object
age_group     object
gender        object
dtype: object

REACTION
Số dòng, số cột: (80, 4)
Kiểu dữ liệu:
patient_id       object
reaction_type    object
severity         object
report_date      object
dtype: object

VACCINATION
Số dòng, số cột: (120, 4)
Kiểu dữ liệu:
patient_id          object
vaccine_type        object
dose_number         object
vaccination_date    object
dtype: object

✅ HOÀN THÀNH NHIỆM VỤ 1
📁 Các file đã được lưu trong thư mục data_clean/


In [2]:
#nhiệm vụ 3
import pandas as pd

patient = pd.read_csv("data_clean/patient_clean.csv")
vaccination = pd.read_csv("data_clean/vaccination_clean.csv")
reaction = pd.read_csv("data_clean/reaction_clean.csv")
#tổng số liều tiêm theo nhóm tuôti và loại vaccin

dose_by_age_vaccine = (
    vaccination
    .merge(patient[["patient_id", "age_group"]], on="patient_id")
    .groupby(["age_group", "vaccine_type"])["dose_number"]
    .count()
    .reset_index(name="total_doses")
)

print("\nTỔNG SỐ LIỀU TIÊM THEO NHÓM TUỔI & LOẠI VACCINE")
print(dose_by_age_vaccine)




TỔNG SỐ LIỀU TIÊM THEO NHÓM TUỔI & LOẠI VACCINE
   age_group vaccine_type  total_doses
0        0_5        D.P.T            0
1        0_5          MMR            1
2      18-49        COVID            1
3      18-49      COVID19            1
4      18-49        D.P.T            1
5      18-49          DPT            1
6      18-49          MMR            0
7      18_49        COVID            2
8      18_49          DPT            1
9      18_49          MMR            0
10       50+      COVID19            1
11       50+        D.P.T            1
12    50PLUS        COVID            1
13    50PLUS      COVID19            1
14    50PLUS          MMR            1
15      6-17        COVID            2
16      6-17          DPT            0
17      6_17        COVID            1
18      6_17      COVID19            1
19      6_17          MMR            1


In [3]:
#trung bình số liều tiêm theo giới tính

dose_per_patient = (
    vaccination
    .groupby("patient_id")["dose_number"]
    .count()
    .reset_index(name="num_doses")
)

avg_dose_by_gender = (
    dose_per_patient
    .merge(patient[["patient_id", "gender"]], on="patient_id")
    .groupby("gender")["num_doses"]
    .mean()
    .reset_index(name="avg_doses")
)

print("\nTRUNG BÌNH SỐ LIỀU TIÊM THEO GIỚI TÍNH")
print(avg_dose_by_gender)



TRUNG BÌNH SỐ LIỀU TIÊM THEO GIỚI TÍNH
  gender  avg_doses
0      F   0.833333
1      M   1.100000
2    NAM   1.000000


In [4]:
#tỉ lrrj bệnh nhân phản ứng sau tiêm

patients_per_vaccine = (
    vaccination
    .groupby("vaccine_type")["patient_id"]
    .nunique()
    .reset_index(name="total_patients")
)

patients_with_reaction = (
    reaction
    .merge(vaccination[["patient_id", "vaccine_type"]], on="patient_id")
    .groupby("vaccine_type")["patient_id"]
    .nunique()
    .reset_index(name="patients_with_reaction")
)

reaction_rate_by_vaccine = (
    patients_per_vaccine
    .merge(patients_with_reaction, on="vaccine_type", how="left")
    .fillna(0)
)

reaction_rate_by_vaccine["reaction_rate (%)"] = (
    reaction_rate_by_vaccine["patients_with_reaction"]
    / reaction_rate_by_vaccine["total_patients"]
    * 100
)

print("\nTỶ LỆ BỆNH NHÂN CÓ PHẢN ỨNG THEO LOẠI VACCINE")
print(reaction_rate_by_vaccine)



TỶ LỆ BỆNH NHÂN CÓ PHẢN ỨNG THEO LOẠI VACCINE
  vaccine_type  total_patients  patients_with_reaction  reaction_rate (%)
0        COVID               7                       7              100.0
1      COVID19               4                       4              100.0
2        D.P.T               3                       3              100.0
3          DPT               3                       3              100.0
4          MMR               8                       8              100.0


In [5]:
#mức độ phản ứng trung bình theo nhóm tuổi
#MỨC ĐỘ PHẢN ỨNG TRUNG BÌNH THEO NHÓM TUỔI
# (số phản ứng trung bình trên mỗi bệnh nhân)

reaction_count = (
    reaction
    .merge(patient[["patient_id", "age_group"]], on="patient_id")
    .groupby(["age_group", "patient_id"])
    .size()
    .reset_index(name="num_reactions")
)

avg_reaction_by_age = (
    reaction_count
    .groupby("age_group")["num_reactions"]
    .mean()
    .reset_index(name="avg_reactions_per_patient")
)

print("\nMỨC ĐỘ PHẢN ỨNG TRUNG BÌNH THEO NHÓM TUỔI")
print(avg_reaction_by_age)




MỨC ĐỘ PHẢN ỨNG TRUNG BÌNH THEO NHÓM TUỔI
  age_group  avg_reactions_per_patient
0       0_5                        2.0
1     18-49                        2.0
2     18_49                        2.0
3       50+                        1.5
4    50PLUS                        1.5
5      6-17                        1.0
6      6_17                        1.0


In [6]:
dose_by_age_vaccine.to_csv("data_clean/dose_by_age_vaccine.csv", index=False)
avg_dose_by_gender.to_csv("data_clean/avg_dose_by_gender.csv", index=False)
reaction_rate_by_vaccine.to_csv("data_clean/reaction_rate_by_vaccine.csv", index=False)
avg_reaction_by_age.to_csv("data_clean/avg_reaction_by_age.csv", index=False)

print("\n✅ HOÀN THÀNH NHIỆM VỤ 3 – ĐÃ LƯU FILE KẾT QUẢ")
## Task 03 – GroupBy (da cap nhat)



✅ HOÀN THÀNH NHIỆM VỤ 3 – ĐÃ LƯU FILE KẾT QUẢ
